In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df  = pd.read_csv('/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv', parse_dates=True)

In [ ]:
df["Date"].unique().shape[0]

In [ ]:
df = df.filter(items=['Date',"SecuritiesCode", 'Target'])
prices_target = df.pivot(index='Date', columns='SecuritiesCode', values='Target')
prices_target.head()

In [ ]:
prices_target_copy=prices_target.copy()

In [ ]:
daily_spread_return=[]
for i in range(1,prices_target.shape[0]+1,1):
    prices_target=prices_target_copy.iloc[0:i]
    
    first_200 = []
    for i in range(0,len(prices_target.index),1):
          first_200.append(-np.sort(-prices_target.iloc[i, :].values)[:200])
    prices_target_first_200 = pd.DataFrame(first_200,index=prices_target.index)
    
    last_200 = []
    for i in range(0,len(prices_target.index),1):
          last_200.append(np.sort(prices_target.iloc[i, :].values)[0:200])
    prices_target_last_200 = pd.DataFrame(last_200,index=prices_target.index)
    weights = np.linspace(start=2, stop=1, num=200)
    Sup=((prices_target_first_200 * weights).sum(axis = 1))/np.mean(weights)
    Sdown=((prices_target_last_200 * weights).sum(axis = 1))/np.mean(weights)
    daily_spread_return += [(Sup - Sdown).mean()/(Sup - Sdown).std()]

In [ ]:
SecuritiesCode_weight=[]
for i in range(1,prices_target.shape[0]+1,1):
    
    prices_target=prices_target_copy.iloc[0:i]
    prices_target=prices_target.replace(np.nan, 100) # get rid of NaN for counting
    count_SecuritiesCode_first_200 = prices_target.isin(prices_target_first_200.values.flatten())
    True_first_200 = (count_SecuritiesCode_first_200.apply(pd.Series.value_counts, axis=0).fillna(0).iloc[1:2]).T
    True_first_200 = True_first_200.add_prefix('first_200_')
    count_SecuritiesCode_last_200 = prices_target.isin(prices_target_last_200.values.flatten())
    True_last_200 = (count_SecuritiesCode_last_200.apply(pd.Series.value_counts, axis=0).fillna(0).iloc[1:2]).T
    True_last_200 = True_last_200.add_prefix('last_200_')
    Count = pd.concat([True_first_200,True_last_200],axis=1)
    SecuritiesCode_weight += [(Count/len(prices_target.index)).sum(axis=1)/2]

In [ ]:
compare=pd.DataFrame(SecuritiesCode_weight)
compare["daily_spread_return"]=daily_spread_return

In [ ]:
Correlation = pd.DataFrame(compare.corr()["daily_spread_return"])

In [ ]:
Correlation.nlargest(5, 'daily_spread_return')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import seaborn as sns

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

p = sns.lineplot(df_weights[Correlation.nlargest(5, 'daily_spread_return').index[1]],daily_spread_return)
p.set_xlabel("Count of SecuritiesCode(2819) (percentage)", fontsize = 20)
p.set_ylabel("daily_spread_return", fontsize = 20)
p.set(title='Correlation of Returns and SecuritiesCode(2819) over increasing time')

In [ ]:
Correlation.nsmallest(5, "daily_spread_return")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

p = sns.lineplot(df_weights[Correlation.nsmallest(5, 'daily_spread_return').index[0]],daily_spread_return)
p.set_xlabel("Count of SecuritiesCode(5020) (percentage)", fontsize = 20)
p.set_ylabel("daily_spread_return", fontsize = 20)
p.set(title='Correlation of Returns and SecuritiesCode(5020) over increasing time')